## New sectioned configuration file help

This document explains how to write a configuration file with sections and subsections, allowing the user to provide their own functions for e.g., the mass-observable relation and its intrinsic scatter

Let's start with the model. This follows the same syntax as before:

```
model            halo.model
```

From now on, things change. We now define *sections*, which refer to each of the
components of the halo model. The order of sections must be followed for the model
to be set up properly, but each component may have a custom number of parameters
(and this is the whole idea behind the new configuration file structure):

```
[section]
[section/subsection1]
parameter1
parameter2
[section/subsection1/subsubsection1]
parameter1
parameter2
[section/subsection2]
parameter1
parameter2
```

etc, etc. This will be enough to explain what's going on (empty lines and lines starting with `'#'` are ignored).

So the first section of the actual configuration file defines cosmological parameters:

```
[cosmo]

sigma_8         fixed     0.8159
H0              fixed     67.74
omegam          fixed     0.3089
omegab_h2       fixed     0.02230
omegav          fixed     0.6911
n               fixed     0.9667
z               fixed     0.188
```

They need not all be fixed, but in this example they are. At any rate, this will always be just a list of parameters, so no major changes compared to the previous version.

Now we move on to the HOD, and this is where the fun begins.

```
[hod]

[hod/observables]
bin_min       10
bin_max       12.5
average       11.315

[hod/ingredients]
centrals       True
satellites     False

[hod/centrals]
fc             uniform        0.1     5     1
bias           fixed          1
[hod/centrals/mor]
name           powerlaw
logM_0         fixed          14.0
a              uniform        -1      1     0
b              uniform        0       2     1
[hod/centrals/scatter]
name           lognormal
sigma_c        uniform        0.05    3     0.5
[hod/centrals/miscentring]
name           fiducial
p_off          uniform        0       1     0.2
R_off          uniform        0       1.5   0.5
```

The whole idea behind this structure is that the HOD may be fully specified by the user, including for instance the complexity of the mass-observable scaling relation. Note that the HOD may also contain a model for satellites and potentially other ingredients, but a simple centrals-only model will serve our purpose here.

We would usually want to have an additional section, `[setup]`, which would include things like the `k`-binning scheme, e.g.,

```
[setup]

lnk_min          -13
lnk_max          13
kbins            1000
```

Finally, any outputs produced by the model, apart from the free parameters. You will usually want to have each ESD component here at the very least. In our case, we'll just output the ESD and the average mass:

```
[output]

esd            8E
Mavg           E
```

And finally the sampler section, which remains the same:

```
[sampler]

path_data           path/to/data
data                shearcovariance_bin_1_A.txt     0,1,4
path_covariance     path/to/covariance
covariance          shearcovariance_matrix_A.txt    4,6

exclude_bins         11,12
sampler_output       output/model.fits
sampler              emcee                    # MCMC sampler
nwalkers             120
nsteps               2000
nburn                0
thin                 1
threads              3
sampler_type         ensemble
update_freq          360
```

## Coding up your own model

With all these sections and parameters, the pipeline can interpret any model passed to it, no matter the number of parameters or the ordering of the ingredients, provided that the configuration file is consistent with the model structure.

For instance, we might define a mass-observable relation with a power-law with mass and redshift,
$$
\log m_\star = A + B\log\left(\frac{M_\mathrm{h}}{M_0}\right) + C\left(\frac{1+z}{1+z_0}\right)\,,
$$
which coded up looks like:

In [7]:
def powerlaw_mz(M, z, A, B, C, logM0, z0):
    return 10**(A + B*(np.log10(M)-logM0) + C*((1+z)/(1+z0)))

Note that these custom functions must include the halo mass as the first argument; all other arguments must be
defined in the configuration file -- including the redshift, in this case. This would therefore require the central MOR section in the configuration file to look like

```
[hod/centrals/mor]
name           powerlaw_mz
z              fixed          0.188
A              uniform        10      16    12
B              uniform        0       5     1
C              uniform        -1      1     0
logM0          fixed          14
z0             fixed          0.1
```

For the time being, you must include your model in the file `kids_ggl_pipeline/halomodel/hod/relations.py`, and custom functions for the scatter about this relation must be included in the file `kids_ggl_pipeline/halomodel/hod/scatter.py`. This has the undesirable effect that this file, common to all users, might get clogged with trial-and-error attempts by various users. We will implement the ability to input user-provided files in the future.

#### Future improvements

 * Choosing between mass-concentration relations should be trivial under this scheme. Should add a subsection `[hod/centrals/concentration]` and possibly one for satellites
 * Custom relations should be written in a user-supplied file rather than in the pipeline source code.